In [12]:
# -*-coding:utf-8-*-

import pymysql
import os
import xlrd

connection = pymysql.connect(host='39.105.9.20', user='root', passwd='bigdata_oil',
                db='cxd_data', port=3306, charset='utf8')
cursor = connection.cursor()

oil_key = ['油', '沥青', '92', '95', '0', '35', '10']
trade_key = ['购', '销']

docxlist = {}
com_name = ''


def gci(filepath):
    # 遍历filepath下所有文件，包括子目录
    global com_name
    files = os.listdir(filepath)
    for fi in files:
        if fi == '.DS_Store':
            continue
        fi_d = os.path.join(filepath, fi)
        if os.path.isdir(fi_d):
            if '公司' in fi:
                com_name = fi
                docxlist[com_name] = []
            gci(fi_d)
        else:
            docxlist[com_name].append(fi_d)
    return docxlist


def read_xls(path):
    # xls文件解析
    record = []
    data = None
    try:
        data = xlrd.open_workbook(path, encoding_override="cp936")  # 打开xls文件
    except Exception as e:
        print(e)
        data = xlrd.open_workbook(path, encoding_override="cp1252")
    finally:
        if data.sheets():
            table = data.sheets()[0]  # 打开第一张表
            nrows = table.nrows  # 获取表的行数
            for i in range(nrows):  # 循环逐行打印
                if i == 0:  # 跳过第一行
                    continue
                record.append(table.row_values(i))
        return record


def data_merge(com, original_data):
    # 将序时账合并为对应的一条数据
    # 样例数据
    # [['10', '*', '', '', '2019-10-31', '记-1', '购货', '140587', '库存商品 - 0号  车用柴油(IV)', '', '', '', '', '',
    # 'RMB', '', 199821.24, 199821.24, 0.0, 'Manager', '', 'Manager', '', '', '', '32.221', '6,201.58'],
    # ['10', '*', '', '', '', '', '购货', '140564', '库存商品 - 0#车用柴油 （VI）', '', '', '', '', '', 'RMB', '',
    # 635065.12, 635065.12, 0.0, 'Manager', '', 'Manager', '', '', '', '100.654', '6,309.39'],
    # ['10', '*', '', '', '', '', '购货', '22210101', '应交税费 - 应交增值税 - 进项税额', '', '', '', '', '', 'RMB', '',
    # 108535.22, 108535.22, 0.0, 'Manager', '', 'Manager', '', '', '', '', ''],
    # ['10', '*', '', '', '', '', '购货', '220240', '应付账款 - 四川中油九洲北斗科技能源有限公司', '', '', '', '', '', 'RMB',
    # '', 943421.58, 0.0, 943421.58, 'Manager', '', 'Manager', '', '', '', '', '']]
    result = []
    oil_list = []
    com_list = []
    if '购' in original_data[0][6]:
        flag = 0    # 0:购 1:销
    else:
        flag = 1
    for record in original_data:
        if any(key in record[8] for key in oil_key) and '公司' not in record[8]:
            for r in record[8].replace(' ', '').split('-'):
                if any(key in r for key in oil_key):
                    oil_list.append(r)
        elif '公司' in record[8]:
            for r in record[8].replace(' ', '').split('-'):
                if '公司' in r:
                    com_list.append(r)
    oil_num = len(oil_list)
    com_num = len(com_list)
    if oil_num == 1 and com_num == 1:   # 油品公司一一对应，最简单的匹配
        if flag == 0:
            xf = com_list[0]
            gf = com
        else:
            xf = com
            gf = com_list[0]
        date = original_data[0][4]
        spmc = oil_list[0]
        Je = original_data[0][16]
        Sl = original_data[0][25] if original_data[0][25] else None
        Dj = original_data[0][26] if original_data[0][26] else None
        result.append((xf, gf, date, spmc, Je, Sl, Dj))
    elif oil_num > 1 and com_num == 1:  # 一个公司多个油品：
        if flag == 0:
            xf = com_list[0]
            gf = com
        else:
            xf = com
            gf = com_list[0]
        date = original_data[0][4]
        for m in oil_list:
            for n in original_data:
                if m in n:
                    spmc = oil_list[0]
                    Je = n[16]
                    Sl = n[25] if n[25] else None
                    Dj = n[26] if n[26] else None
                    result.append((xf, gf, date, spmc, Je, Sl, Dj))
    else:   # 多个油品对应多家公司，油品数目与公司数目可能不等
        print('暂时没想到好方法,可先将此文件做记录，后续处理')
        return -1
    return result


if __name__ == "__main__":
    """
    序时账表头
    ['0期间', '1过账', '2审核', '3作废', '4日期', '5凭证字号', '6摘要', '7科目代码', '8科目名称', 
    '9往来单位', '10部门', '11职员', '12自定义项目', '13业务编号', '14币别', '15汇率', '16原币金额', 
    '17借方金额', '18贷方金额', '19制单人', '20审核人', '21过账人', '22结算方式', '23结算号',
     '24结算日期', '25数量', '26单价', '27附件数']
    """
    path_list = gci('XSZ')
    match_error_list = []
    update_list = []
    error_list = []
    for k, v in path_list.items():
        for file_path in v:
            final_data = []
            if '序时' not in file_path:
                continue
            print(file_path)
            file_data = read_xls(file_path)

            if file_data:
                # 跳过无关公司的序时账
                if len(file_data[0]) < 27:
                    continue
                if '2019' in str(file_data[0][4]):
                    temp = []
                    temp_data = []
                    for r in file_data:
                        if temp and r[4]:   # 判断是否是新账目
                            if any(key in temp[0][6] for key in trade_key):     # 判断是否是购销
                                if any(key in temp[0][8] for key in oil_key) and '公司' not in temp[0][8]:    # 是否包含油品
                                    temp_data.append(temp)
                            temp = []
                        temp.append(r)
                    for r in temp_data:
                        try:
                            middle_data = data_merge(k, r)
                            if middle_data ==-1:
                                match_error_list.append(file_path) # 添加没想到好方法文件的路径信息
                                continue
                            if middle_data:
                                for j in middle_data:
                                    final_data.append(j)
                        except Exception as e:
                            print('有问题公司：', file_path)
                            print(str(e))
                            error_list.append(file_path) # 添加解析文件出错的文件信息
#             print(final_data)
            update_list.extend(final_data)
#     print('err:',error_list)
#     print('update:',update_list)



XSZ\2019-11-22\世旺(大连）石油化工有限公司\ea85215b-b8d0-4109-a81a-efe1edd8222b_世旺1-10月序时账.xls
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
暂时没想到好方法,可先将此文件做记录，后续处理
暂时没想到好方法,可先将此文件做记录，后续处理
暂时没想到好方法,可先将此文件做记录，后续处理
XSZ\2019-11-22\世纪永华实业（大连）有限公司\e64dfe70-57f1-410c-ac4f-3b390e42df5b_世纪永华实业序时账.xls
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
XSZ\2019-11-22\中东港湾国际贸易（大连）有限公司\b88008d5-e35b-46cf-b9bd-98908dbb3dd5_中东港湾国际贸易（大连）有限公司序时簿.xls
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
XSZ\2019-11-22\中东荣信（大连）石油有限公司\e0bb68bc-4310-4857-9ed2-0faabfd1b354_中东荣信1-10月序时账.xls
XSZ\2019-11-22\中亚石油（大连）有限公司\85cace47-06ea-49eb-beb0-466cbc9d2692_序时账中亚石油（大连）有限公司.xls
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
XSZ\2019-11-22\中交建锦沣能源（大连）有限公司\c5743343-3e94-4710-8842-3794034a95c9_中交建序时账.xls
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
XSZ\2019-11-22\中化海洋能源（大连）有限公司\b5dbf15e-1cb1-4c0b-93c8-e5201

暂时没想到好方法,可先将此文件做记录，后续处理
暂时没想到好方法,可先将此文件做记录，后续处理
暂时没想到好方法,可先将此文件做记录，后续处理
XSZ\2019-11-22\中油凯胜源（大连）石油化工有限公司\d30febce-014b-463b-a14e-f893a7e03104_序时账中油凯胜源（大连）石油化工有限公司.xls
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
XSZ\2019-11-22\中油凯迪石油化工（大连）有限公司\0cb7e1d0-58ad-4cf9-a009-33c56e616dbd_中油凯迪石油化工（大连）有限公司序时簿.xls
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
XSZ\2019-11-22\中油加仑多石化（大连）有限公司\b8ec2413-639c-4774-b558-5ecb2b139894_序时账中油加仑多石化（大连）有限公司.xls
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
暂时没想到好方法,可先将此文件做记录，后续处理
暂时没想到好方法,可先将此文件做记录，后续处理
暂时没想到好方法,可先将此文件做记录，后续处理
暂时没想到好方法,可先将此文件做记录，后续处理
暂时没想到好方法,可先将此文件做记录，后续处理
暂时没想到好方法,可先将此文件做记录，后续处理
XSZ\2019-11-22\中油勝达能源（大连）有限公司\3ec12885-6829-4055-ad40-c8e94d611bab_中油勝达能源（大连）有限公司序时账.xls
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
XSZ\2019-11-22\中油北方华油（大连）石化有限公司\da43cfb2-2541-44bb-a501-d9f0ac7e9292_201901-10中油北方华油（大连）石化有限公司序时账.xls
WARNING *** OLE2 incon

XSZ\2019-11-22\中油天莹能源（大连）有限公司\18c17adc-e3c6-4eda-becd-3aea8d37ce28_天莹1-10月序时账.xls
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
暂时没想到好方法,可先将此文件做记录，后续处理
暂时没想到好方法,可先将此文件做记录，后续处理
暂时没想到好方法,可先将此文件做记录，后续处理
暂时没想到好方法,可先将此文件做记录，后续处理
暂时没想到好方法,可先将此文件做记录，后续处理
暂时没想到好方法,可先将此文件做记录，后续处理
暂时没想到好方法,可先将此文件做记录，后续处理
暂时没想到好方法,可先将此文件做记录，后续处理
XSZ\2019-11-22\中油天鸿石油化工（大连）有限公司\a9abc233-c006-498d-8a74-3d0ba5c0a072_会计分录序时簿中油天鸿石油化工（大连）有限公司.xls
XSZ\2019-11-22\中油宇坤石化（大连）有限公司\35cd233e-e219-49d7-a728-5e554aa8b640_中油宇坤石化（大连）有限公司序时账.xls
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
暂时没想到好方法,可先将此文件做记录，后续处理
XSZ\2019-11-22\中油安能（大连）石化有限公司\c7bf9756-8737-4a18-85be-c3da635630a4_中油安能（大连）石化有限公司序时账.xls
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
XSZ\2019-11-22\中油宏伟（大连）石油化工有限公司\b28b77e8-5211-4afa-be21-4780afe0ba61_2019中油宏伟（大连）石油化工有限公司序时账.xls
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
XSZ\2019-11-22\中油宏源石化（大连）有限公司\68f3a246

暂时没想到好方法,可先将此文件做记录，后续处理
暂时没想到好方法,可先将此文件做记录，后续处理
暂时没想到好方法,可先将此文件做记录，后续处理
暂时没想到好方法,可先将此文件做记录，后续处理
暂时没想到好方法,可先将此文件做记录，后续处理
暂时没想到好方法,可先将此文件做记录，后续处理
暂时没想到好方法,可先将此文件做记录，后续处理
暂时没想到好方法,可先将此文件做记录，后续处理
暂时没想到好方法,可先将此文件做记录，后续处理
暂时没想到好方法,可先将此文件做记录，后续处理
暂时没想到好方法,可先将此文件做记录，后续处理
暂时没想到好方法,可先将此文件做记录，后续处理
暂时没想到好方法,可先将此文件做记录，后续处理
暂时没想到好方法,可先将此文件做记录，后续处理
暂时没想到好方法,可先将此文件做记录，后续处理
暂时没想到好方法,可先将此文件做记录，后续处理
XSZ\2019-11-22\中油泰和（大连）石油天然气有限公司\e7e64d59-910d-4f30-bf5d-b729a48fb1c8_泰和序时账.xls
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
XSZ\2019-11-22\中油泽锜石化（大连）有限公司\fbc6231d-c887-4399-bccd-68d418f330ba_中油泽锜石化（大连）有限公司序时簿.xls
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
XSZ\2019-11-22\中油浩霖（大连）石化有限公司\48891d6d-078d-49fe-a833-71b4155006b2_浩霖1-10月序时账.xls
XSZ\2019-11-22\中油海洋石油（大连）有限公司\19f9f2f7-92f3-4a3e-ac21-dd49b950c5f7_中油海洋石油（大连）有限公司序时账.xls
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
暂时没想到好方法,可先将此文件做记录，后续处理
暂时没想到好方法,可先将此文件做记录，后续处理
暂时没想到好方法,

暂时没想到好方法,可先将此文件做记录，后续处理
暂时没想到好方法,可先将此文件做记录，后续处理
暂时没想到好方法,可先将此文件做记录，后续处理
暂时没想到好方法,可先将此文件做记录，后续处理
暂时没想到好方法,可先将此文件做记录，后续处理
暂时没想到好方法,可先将此文件做记录，后续处理
暂时没想到好方法,可先将此文件做记录，后续处理
暂时没想到好方法,可先将此文件做记录，后续处理
暂时没想到好方法,可先将此文件做记录，后续处理
暂时没想到好方法,可先将此文件做记录，后续处理
暂时没想到好方法,可先将此文件做记录，后续处理
暂时没想到好方法,可先将此文件做记录，后续处理
暂时没想到好方法,可先将此文件做记录，后续处理
暂时没想到好方法,可先将此文件做记录，后续处理
暂时没想到好方法,可先将此文件做记录，后续处理
暂时没想到好方法,可先将此文件做记录，后续处理
暂时没想到好方法,可先将此文件做记录，后续处理
暂时没想到好方法,可先将此文件做记录，后续处理
暂时没想到好方法,可先将此文件做记录，后续处理
暂时没想到好方法,可先将此文件做记录，后续处理
暂时没想到好方法,可先将此文件做记录，后续处理
暂时没想到好方法,可先将此文件做记录，后续处理
暂时没想到好方法,可先将此文件做记录，后续处理
暂时没想到好方法,可先将此文件做记录，后续处理
暂时没想到好方法,可先将此文件做记录，后续处理
暂时没想到好方法,可先将此文件做记录，后续处理
暂时没想到好方法,可先将此文件做记录，后续处理
暂时没想到好方法,可先将此文件做记录，后续处理
暂时没想到好方法,可先将此文件做记录，后续处理
暂时没想到好方法,可先将此文件做记录，后续处理
暂时没想到好方法,可先将此文件做记录，后续处理
暂时没想到好方法,可先将此文件做记录，后续处理
暂时没想到好方法,可先将此文件做记录，后续处理
XSZ\2019-11-22\中油鑫洋（大连）石油化工有限公司\0a8248af-55e5-465e-a14a-d55c3e29db32_序时簿.xls
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
暂时没想到好方法,可先将此文件做记录，后续处理
XSZ\2019-11-22\中油鑫盛石油化工(大连）有限公司\e1

...

WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
XSZ\2019-11-26\中油宝麟（大连）石化有限公司\520aafbf-00c5-41c3-a5a2-110fd58c3de3_中油天禹（大连）石油化工有限公司序时账.xls
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
XSZ\2019-11-26\中油富杰石油化工（大连）有限公司\38cfc0f8-1a46-4875-9eb5-7a36583f570b_序时账中油富杰石油化工（大连）有限公司.xls
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
XSZ\2019-11-26\中油尊胜石油化工（大连）有限公司\d07a0f9a-7c6c-4592-b399-16dd99ae7603_序时账.xls
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
XSZ\2019-11-26\中油弘汇新石油化工（大连）有限公司\bddf745f-9c65-4c8e-899b-483341ef137f_中油弘汇新石油化工（大连）有限公司1-10月序时账.xls
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
XSZ\2019-11-26\中油恒启石化（大连）有限责任公司\eaae0bf5-e326-437a-b2a0-0d1a02c56a06_恒启7-10序时账.xls
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
暂时没想到好方法,可先将此文件做记录，后续处理
XSZ\2019-11-26\中油恒荣石油化工（大连）有限公司\4a68ce85-860a-4e04-a1fa-f39a1c7b0b2d_恒荣1-10序时账.xls
WARNING ***

XSZ\2019-11-26\中油西太（大连）石油化工有限公司\71fd6b54-e01f-4a42-a27d-e56865c96ec6_中油西太序时账.xls
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
暂时没想到好方法,可先将此文件做记录，后续处理
XSZ\2019-11-26\中油诺威森石化（大连）有限公司\ea9ebf81-a026-4b37-977f-483f226988c4_中油诺威森石化（大连）有限公司1-10月序时账.xls
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
XSZ\2019-11-26\中油豪特（大连）石油化工有限公司\d3e09865-3e9a-4616-b618-59d4cd697030_中油豪特（大连）石油化工有限公司序时账.xls
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
暂时没想到好方法,可先将此文件做记录，后续处理
XSZ\2019-11-26\中油金瑞能源（大连）有限公司\1afea7b6-e17c-42f3-9977-b8fd4706204c_蓝奥1-10序时账.xls
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
XSZ\2019-11-26\中油鑫企源(大连)石油化工有限公司\3b4796ab-37d5-40ef-a8ff-0b7a4aa4cf07_序时簿.xls
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
XSZ\2019-11-26\中油鑫汇（大连）石化有限公司\1bada01d-c9bb-4dca-99ac-28da7428386c_中油鑫汇（大连）石化有限公司1-10月序时账.xls
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is 

XSZ\2019-11-26\大连冀疆石油有限公司\4796d601-41bb-4a33-83b6-80a6ae7b81ae_大连冀疆石油有限公司1-10月序时账.xls
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
XSZ\2019-11-26\大连冠油石油化工有限公司\b9323e5f-2b25-4265-9e90-870c00f14c32_序时账大连冠油石油化工有限公司.xls
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
XSZ\2019-11-26\大连力威石化有限公司\01b0d4da-4df2-4f60-a60c-1f20311a8b92_大连力威石化有限公司序时账.xls
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
暂时没想到好方法,可先将此文件做记录，后续处理
暂时没想到好方法,可先将此文件做记录，后续处理
暂时没想到好方法,可先将此文件做记录，后续处理
暂时没想到好方法,可先将此文件做记录，后续处理
暂时没想到好方法,可先将此文件做记录，后续处理
暂时没想到好方法,可先将此文件做记录，后续处理
暂时没想到好方法,可先将此文件做记录，后续处理
暂时没想到好方法,可先将此文件做记录，后续处理
暂时没想到好方法,可先将此文件做记录，后续处理
暂时没想到好方法,可先将此文件做记录，后续处理
暂时没想到好方法,可先将此文件做记录，后续处理
暂时没想到好方法,可先将此文件做记录，后续处理
暂时没想到好方法,可先将此文件做记录，后续处理
暂时没想到好方法,可先将此文件做记录，后续处理
暂时没想到好方法,可先将此文件做记录，后续处理
暂时没想到好方法,可先将此文件做记录，后续处理
XSZ\2019-11-26\大连力越石化有限公司\d219f07d-1f17-4352-9817-4b18427f68a6_力越序时账.xls
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero

XSZ\2019-11-26\大连海联石油化工有限公司\414a27b1-f48b-4ce8-8a41-39e6985f83c1_大连海联石油化工有限公司1-10序时簿.xls
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
XSZ\2019-11-26\大连海通石油化工有限公司\06b10612-852c-469b-a401-edced300476c_大连海通石油化工有限公司1-10月序时账.xls
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
XSZ\2019-11-26\大连淡水前通石油化工有限公司\b0d52b04-c9c5-4731-93d1-de238347191d_序时账.xls
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
暂时没想到好方法,可先将此文件做记录，后续处理
XSZ\2019-11-26\大连湘联石油化工有限公司\a7ffee97-1507-4cc3-a2cb-9e4e2bdb4c0d_序时簿.xls
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
XSZ\2019-11-26\大连滨海启航科技有限公司\77cafb5d-1678-47f6-afc1-9382154c2eea_大连滨海启航科技有限公司序时账.xls
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
暂时没想到好方法,可先将此文件做记录，后续处理
暂时没想到好方法,可先将此文件做记录，后续处理
暂时没想到好方法,可先将此文件做记录，后续处理
暂时没想到好方法,可先将此文件做记录，后续处理
暂时没想到好方法,可先将此文件做记录，后续处理
XSZ\2019-11-26\大连瑞益力商贸有限公司\9ce287f6-8e73-4fa0-83eb-a43acf3bbb24_大连瑞益力商贸有限公司序时簿2

暂时没想到好方法,可先将此文件做记录，后续处理
暂时没想到好方法,可先将此文件做记录，后续处理
暂时没想到好方法,可先将此文件做记录，后续处理
暂时没想到好方法,可先将此文件做记录，后续处理
暂时没想到好方法,可先将此文件做记录，后续处理
暂时没想到好方法,可先将此文件做记录，后续处理
暂时没想到好方法,可先将此文件做记录，后续处理
暂时没想到好方法,可先将此文件做记录，后续处理
暂时没想到好方法,可先将此文件做记录，后续处理
暂时没想到好方法,可先将此文件做记录，后续处理
暂时没想到好方法,可先将此文件做记录，后续处理
暂时没想到好方法,可先将此文件做记录，后续处理
暂时没想到好方法,可先将此文件做记录，后续处理
暂时没想到好方法,可先将此文件做记录，后续处理
暂时没想到好方法,可先将此文件做记录，后续处理
暂时没想到好方法,可先将此文件做记录，后续处理
暂时没想到好方法,可先将此文件做记录，后续处理
暂时没想到好方法,可先将此文件做记录，后续处理
暂时没想到好方法,可先将此文件做记录，后续处理
暂时没想到好方法,可先将此文件做记录，后续处理
暂时没想到好方法,可先将此文件做记录，后续处理
暂时没想到好方法,可先将此文件做记录，后续处理
暂时没想到好方法,可先将此文件做记录，后续处理
暂时没想到好方法,可先将此文件做记录，后续处理
暂时没想到好方法,可先将此文件做记录，后续处理
暂时没想到好方法,可先将此文件做记录，后续处理
暂时没想到好方法,可先将此文件做记录，后续处理
暂时没想到好方法,可先将此文件做记录，后续处理
暂时没想到好方法,可先将此文件做记录，后续处理
暂时没想到好方法,可先将此文件做记录，后续处理
暂时没想到好方法,可先将此文件做记录，后续处理
暂时没想到好方法,可先将此文件做记录，后续处理
暂时没想到好方法,可先将此文件做记录，后续处理
暂时没想到好方法,可先将此文件做记录，后续处理
暂时没想到好方法,可先将此文件做记录，后续处理
暂时没想到好方法,可先将此文件做记录，后续处理
暂时没想到好方法,可先将此文件做记录，后续处理
暂时没想到好方法,可先将此文件做记录，后续处理
暂时没想到好方法,可先将此文件做记录，后续处理
暂时没想到好方法,可先将此文件做记录，后续处理
暂时没想到好方法,可先将此文件做记录，后续处理
暂时没想到好方法,可先将此文件做

In [13]:
update_list

[('中国石油天然气股份有限公司湖北十堰销售分公司',
  '世纪永华实业（大连）有限公司',
  '2019/04/30',
  '0号车用柴油（VI）',
  494513.27,
  '88.00',
  '5,619.47'),
 ('中国石油天然气股份有限公司湖北十堰销售分公司',
  '世纪永华实业（大连）有限公司',
  '2019/06/30',
  '0号车用柴油（VI）',
  558407.07,
  '100.00',
  '5,584.07'),
 ('中国石油天然气股份有限公司湖北十堰销售分公司',
  '世纪永华实业（大连）有限公司',
  '2019/07/31',
  '0号车用柴油（VI）',
  416902.65,
  '70.00',
  '5,955.75'),
 ('中国石油天然气股份有限公司湖北十堰销售分公司',
  '世纪永华实业（大连）有限公司',
  '2019/09/30',
  '0号车用柴油（VI）',
  24460.18,
  '4.00',
  '6,115.05'),
 ('天津港保税区东方五洲石油化工有限公司',
  '中东港湾国际贸易（大连）有限公司',
  '2019-01-31',
  '沥青',
  18155172.39,
  '4,500.00',
  '4,034.48'),
 ('上海京盛石油化工有限公司',
  '中东港湾国际贸易（大连）有限公司',
  '2019-01-31',
  '0号车用柴油V',
  2283556.03,
  '434.25',
  '5,258.62'),
 ('上海集通石油化工有限公司',
  '中东港湾国际贸易（大连）有限公司',
  '2019-01-31',
  '沥青',
  984224.14,
  '245.00',
  '4,017.24'),
 ('上海京盛石油化工有限公司',
  '中东港湾国际贸易（大连）有限公司',
  '2019-01-31',
  '沥青',
  12838271.58,
  '3,241.00',
  '3,961.21'),
 ('天津港保税区东方五洲石油化工有限公司',
  '中东港湾国际贸易（大连）有限公司',
  '2019-03-31',
  '沥青',
  20818965.51,
  '5

In [14]:
match_error_list

['XSZ\\2019-11-22\\世旺(大连）石油化工有限公司\\ea85215b-b8d0-4109-a81a-efe1edd8222b_世旺1-10月序时账.xls',
 'XSZ\\2019-11-22\\世旺(大连）石油化工有限公司\\ea85215b-b8d0-4109-a81a-efe1edd8222b_世旺1-10月序时账.xls',
 'XSZ\\2019-11-22\\世旺(大连）石油化工有限公司\\ea85215b-b8d0-4109-a81a-efe1edd8222b_世旺1-10月序时账.xls',
 'XSZ\\2019-11-22\\中化海洋能源（大连）有限公司\\b5dbf15e-1cb1-4c0b-93c8-e52015b9b577_序时账中化海洋能源（大连）有限公司.xls',
 'XSZ\\2019-11-22\\中化海洋能源（大连）有限公司\\b5dbf15e-1cb1-4c0b-93c8-e52015b9b577_序时账中化海洋能源（大连）有限公司.xls',
 'XSZ\\2019-11-22\\中化海洋能源（大连）有限公司\\b5dbf15e-1cb1-4c0b-93c8-e52015b9b577_序时账中化海洋能源（大连）有限公司.xls',
 'XSZ\\2019-11-22\\中化海洋能源（大连）有限公司\\b5dbf15e-1cb1-4c0b-93c8-e52015b9b577_序时账中化海洋能源（大连）有限公司.xls',
 'XSZ\\2019-11-22\\中化海洋能源（大连）有限公司\\b5dbf15e-1cb1-4c0b-93c8-e52015b9b577_序时账中化海洋能源（大连）有限公司.xls',
 'XSZ\\2019-11-22\\中化海洋能源（大连）有限公司\\b5dbf15e-1cb1-4c0b-93c8-e52015b9b577_序时账中化海洋能源（大连）有限公司.xls',
 'XSZ\\2019-11-22\\中化海洋能源（大连）有限公司\\b5dbf15e-1cb1-4c0b-93c8-e52015b9b577_序时账中化海洋能源（大连）有限公司.xls',
 'XSZ\\2019-11-22\\中化海洋能源（大连）有限公司\\b5dbf15e-1cb1-4c0b-93c8-e5

In [15]:
import json

list_final = []
for i in match_error_list:
    if i not in list_final:
        list_final.append(i)
    else:
        continue
        
print(len(list_final))

list_final

414


['XSZ\\2019-11-22\\世旺(大连）石油化工有限公司\\ea85215b-b8d0-4109-a81a-efe1edd8222b_世旺1-10月序时账.xls',
 'XSZ\\2019-11-22\\中化海洋能源（大连）有限公司\\b5dbf15e-1cb1-4c0b-93c8-e52015b9b577_序时账中化海洋能源（大连）有限公司.xls',
 'XSZ\\2019-11-22\\中化润德能源（大连）有限公司\\87358889-8a6e-4b87-86da-53be6a1dc5ae_201901-10中化润德能源（大连）有限公司序时账.xls',
 'XSZ\\2019-11-22\\中投珺泽石化（大连）有限公司\\5ca56c9c-f55c-4246-95eb-123f22b210d8_中投珺泽序时账.xls',
 'XSZ\\2019-11-22\\中昱能源（大连）有限公司\\5e4a16e7-047e-43be-8896-5e2666e51037_中昱序时账.xls',
 'XSZ\\2019-11-22\\中油万邦石油化工（大连）有限公司\\093440f4-f853-4143-a206-9f9dfeec32ce_中油万邦石油化工（大连）有限公司序时簿.xls',
 'XSZ\\2019-11-22\\中油东成石油化工（大连）有限公司\\e94b8c6b-65be-4a2f-b92c-597e3b129b93_序时账中油东成石油化工（大连）有限公司.xls',
 'XSZ\\2019-11-22\\中油东胜石油（大连）有限公司\\06fe71d8-b2e2-40b9-9f7c-7c28448ef7e3_序时账中油东胜石油（大连）有限公司.xls',
 'XSZ\\2019-11-22\\中油中燃石油化工（大连）有限公司\\3c98c291-ae98-4f19-aba0-e6df3d3e9594_序时账中油中燃石油化工（大连）有限公司.xls',
 'XSZ\\2019-11-22\\中油中胜（大连）石油化工有限公司\\767b8b3f-5141-436b-80fa-52320fdea8ae_2019中油中胜（大连）石油化工有限公司序时账.xls',
 'XSZ\\2019-11-22\\中油丰泰能源（大连）有限公司\\8556ff4

In [16]:
# 上传到数据库
# 交易表入库sql
trade_sql = "INSERT INTO financial_exchange (Xf_company_name,Gf_company_name,exchange_date,exchange_good,Je,Sl,Dj" \
            ") VALUES (%s,%s,%s,%s,%s,%s,%s)"
cursor.executemany(trade_sql, update_list)
connection.commit()

D:\Selvaria\Anaconda\lib\site-packages\pymysql\cursors.py:165: Warning: (1265, "Data truncated for column 'Dj' at row 1")
  result = self._query(query)
D:\Selvaria\Anaconda\lib\site-packages\pymysql\cursors.py:165: Warning: (1265, "Data truncated for column 'Dj' at row 2")
  result = self._query(query)
D:\Selvaria\Anaconda\lib\site-packages\pymysql\cursors.py:165: Warning: (1265, "Data truncated for column 'Dj' at row 3")
  result = self._query(query)
D:\Selvaria\Anaconda\lib\site-packages\pymysql\cursors.py:165: Warning: (1265, "Data truncated for column 'Dj' at row 4")
  result = self._query(query)
D:\Selvaria\Anaconda\lib\site-packages\pymysql\cursors.py:165: Warning: (1265, "Data truncated for column 'Sl' at row 5")
  result = self._query(query)
D:\Selvaria\Anaconda\lib\site-packages\pymysql\cursors.py:165: Warning: (1265, "Data truncated for column 'Dj' at row 5")
  result = self._query(query)
D:\Selvaria\Anaconda\lib\site-packages\pymysql\cursors.py:165: Warning: (1265, "Data tru

D:\Selvaria\Anaconda\lib\site-packages\pymysql\cursors.py:165: Warning: (1265, "Data truncated for column 'Sl' at row 29")
  result = self._query(query)
D:\Selvaria\Anaconda\lib\site-packages\pymysql\cursors.py:165: Warning: (1265, "Data truncated for column 'Sl' at row 40")
  result = self._query(query)
D:\Selvaria\Anaconda\lib\site-packages\pymysql\cursors.py:165: Warning: (1265, "Data truncated for column 'Sl' at row 46")
  result = self._query(query)
D:\Selvaria\Anaconda\lib\site-packages\pymysql\cursors.py:165: Warning: (1265, "Data truncated for column 'Sl' at row 48")
  result = self._query(query)
D:\Selvaria\Anaconda\lib\site-packages\pymysql\cursors.py:165: Warning: (1265, "Data truncated for column 'Sl' at row 54")
  result = self._query(query)
D:\Selvaria\Anaconda\lib\site-packages\pymysql\cursors.py:165: Warning: (1265, "Data truncated for column 'Sl' at row 55")
  result = self._query(query)
D:\Selvaria\Anaconda\lib\site-packages\pymysql\cursors.py:165: Warning: (1265, "Da